- - - 
MODEL GENERATION FROM SCRATCH

Generation of a new model from scratch, including database and data input

(settings -> model -> use_existing_data -> False)

In [ ]:
# create model directory with essential setup files
# to be used only to generate models from scratch
from src.frontend import esm

model_dir_name = '0_hsut'
main_dir_path = 'D:\Politecnico di Milano\Documentale DENG - PRIN-MIMO\Models\pyESM_docs\case_studies'

esm.create_model_dir(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    force_overwrite=True,
)

In [ ]:
# generate model instance based on setup files filled by the user.
# validates model directory (in case user relies on existing directory). 
# parse settings and paths.
from src.frontend import esm

model_dir_name = '0_hsut'
main_dir_path = 'D:\Politecnico di Milano\Documentale DENG - PRIN-MIMO\Models\pyESM_docs\case_studies'

model = esm.Model(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    log_level='debug',
)

In [ ]:
# after filling sets.xlsx file: 
# - loading model coordinates to Index
# - generating sqlite database tables for sets and variables
# (variables tables in sqlite database empty)
# (error occurring at the first run related to foreign_keys. second run works)
model.load_model_coordinates()
model.initialize_blank_database()

In [ ]:
# after filling input_data file/s:
# - loading input data into sqlite database variables tables
# - initialize problem
model.load_data_files_to_database()
model.initialize_problems()

- - - 
MODEL GENERATION FROM EXISTING DATA

Generation of a new model working with existing database and data input files.

(settings -> model -> use_existing_data -> True)

In [1]:
# generate model instance based on setup files filled by the user.
# validates model directory (in case user relies on existing directory). 
# parse settings and paths.
# loading model coordinates
# initializing numerical problem
from src.frontend import esm

model_dir_name = '0_hsut'
main_dir_path = 'D:\Politecnico di Milano\Documentale DENG - PRIN-MIMO\Models\pyESM_docs\case_studies'

model = esm.Model(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    log_level='debug',
)

INFO | Model | 'Model' object initialization...
INFO | Model.file_manager | 'FileManager' object generated.
INFO | Model | Model directory and required setup files exist.
INFO | Model | Loading settings.
DEBUG | Model.file_manager | File 'settings.yml' loaded.
INFO | Model | Loading paths from settings.
INFO | Model.core | 'Core' object initialization...
INFO | Model.core.sql_manager | 'SQLManager' object generation.
INFO | Model.core.index | 'Index' object initialization...
DEBUG | Model.file_manager | File 'sets_structure.yml' loaded.
DEBUG | Model.file_manager | File 'variables.yml' loaded.
DEBUG | Model.core.index | Loading 'variables_fields' to Index.
DEBUG | Model.core.index | Loading variables 'table_headers' to Index.
DEBUG | Model.core.index | Loading variables 'sets_parsing_hierarchy' to Index.
INFO | Model.core.index | 'Index' object initialized.
INFO | Model.core.database | 'Database' object initialization...
INFO | Model.core.database | 'Database' object initialized.
INFO 

- - -
MODEL USE 

solving numerical model, results export to sqlite database, generation of powerbi
dataset

In [ ]:
# in case of modifications in input data files (not in sets),
# update database and problem
model.update_database_and_problem()

In [2]:
# solve numerical problems
model.solve_problem()

INFO | Model | Solving numerical problem.
INFO | Model.core | Solve numerical problems.
DEBUG | Model.core.problem | Solving problem: ['Reference', 'min']
c:\ProgramData\Anaconda3\envs\esm\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
DEBUG | Model.core.problem | Solving problem: ['Reference', 'max']
DEBUG | Model.core.problem | Solving problem: ['NetZero', 'min']
DEBUG | Model.core.problem | Solving problem: ['NetZero', 'max']


In [3]:
# once model has successfully solved, load endogenous parameters data to 
# sqlite database and generate powerbi dataset.
model.load_results_to_database()
model.generate_pbi_report()

INFO | Model | Exporting endogenous model results to SQLite database.
DEBUG | Model.core.sql_manager | Connection to 'database.db' opened.
INFO | Model.core | Fetching data from cvxpy endogenous variables to database 'database.db' 
DEBUG | Model.core | Fetching data from cvxpy variable 'Q to the related SQLite table.
DEBUG | Model.core.sql_manager | SQLite table 'Q' - 36 entries deleted.
DEBUG | Model.core.sql_manager | SQLite table 'Q' - table overwritten and 36 entries added.
DEBUG | Model.core | Fetching data from cvxpy variable 'X to the related SQLite table.
DEBUG | Model.core.sql_manager | SQLite table 'X' - 36 entries deleted.
DEBUG | Model.core.sql_manager | SQLite table 'X' - table overwritten and 36 entries added.
DEBUG | Model.core.sql_manager | Connection to 'database.db' closed.
INFO | Model | Generating PowerBI report.
DEBUG | Model.pbi_manager | PowerBI report file 'dataset.pbix' created.


- - - 
TESTS

In [ ]:
import pandas as pd


var_test = model.core.index.variables.Q
df_test = pd.DataFrame()

for row in var_test.data.index:
    df_test = pd.concat(
        (df_test, var_test.reshaping_variable_data(row))
    )

var_test.reshaping_variable_data(1)
df_test

In [ ]:
model.core.index.sets.years.data

In [ ]:
model.core.index.variables.u.data


In [ ]:
model.core.problem.symbolic_problem

In [ ]:
model.core.index.variables.X.data.at[0, 'variable'].value


In [ ]:
if model.core.problem.numeric_problems is None:
    print('ciao')

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
# b = np.random.randn(m)
b = 1

# Construct the problem.
x = cp.Variable((n,1))
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.

x.value

In [ ]:
# DEFINIZIONE COSTANTI PER TRASFORMARE I VARI OPERATORI VETTORIALI
import cvxpy as cp
import numpy as np

var_a = cp.Variable((3,2))
var_a.value = np.ones((var_a.shape))*3
var_b = cp.Variable((3,1))

i_21 = cp.Constant(np.ones((2,1)))
i_13 = cp.Constant(np.ones((1,3)))

allowed_vars = {
    'A': var_a,
    'B': var_b,
    'i_21': i_21,
    'i_13': i_13,
}

allowed_ops = {
    '+': '+',
    '-': '-',
    '*': '*',
    '@': '@',
    '==': '==',
    '>=': '>=',
    '<=': '<=',
    '(': '(',
    ')': ')',
    ',':',',
    'sum': cp.sum,
}

expr = model.core.problem.execute_cvxpy_code(
    expression='i_13 @ A @ i_21',
    allowed_variables=allowed_vars,
    allowed_operators=allowed_ops
)

expr2 = model.core.problem.execute_cvxpy_code(
    expression='sum(B)',
    allowed_variables=allowed_vars,
    allowed_operators=allowed_ops
)

expr2

In [ ]:
import pandas as pd
from src.util import constants
from src.util import util

field_id = constants._STD_ID_FIELD['id'][0]

col_to_update = constants._STD_VALUES_FIELD['values'][0]
cols_common = list(variable.coordinates.keys())

new_df = variable.reshaping_variable_data(0)

new_values = {
    's_Name': 'Reference',
    'dt_Name': '2023',
    't_Name': 'Final demand',
    'values': 15.0
}

new_df.loc[len(new_df)] = new_values
new_df.at[0,'values'] = 2

test.core.sqltools.open_connection()
existing_df = test.core.sqltools.table_to_dataframe(table_name=variable.symbol)
existing_df.drop(columns=['id'],inplace=True)
test.core.sqltools.close_connection()

print('\n')
print('existing_df: \n', existing_df)
print('\n')
print('new_df: \n', new_df)

In [ ]:
variable=test.core.index.variables.u

test.core.database.sqltools.open_connection()
raw_data = test.core.database.sqltools.filtered_table_to_dataframe(
    table_name=variable.symbol,
    filters_dict=variable.data['filter'][0]
)
test.core.database.sqltools.close_connection()

raw_data

In [ ]:
test.core.database.sqltools.open_connection()

var = test.core.database.sqltools.filtered_table_to_dataframe(
    table_name='u',
    filters_dict={
        's_Name': ['Reference'],
        'dt_Name': [2023],
        'f_Name': ['Energy', 'Steel'],
        't_Name': ['PV power plant', 'Steel factory', 'Gas power plant']
    }
)

# filter_1 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='f_Name',
#     parent_table_name='_set_FLOWS',
#     parent_table_fields={
#         'f_Category': ['Product flow'],
#     }
# )

# filter_2 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='dt_Name',
#     parent_table_name='_set_DATETIME',
#     parent_table_fields={
#         'dt_Name': [2023],
#     }
# )

# var_custom_filter = test.core.database.sqltools.filtered_table_to_dataframe(
#     table_name='v',
#     filters_dict={
#         **filter_1,
#         **filter_2,
#         **{'f_Name': ['Steel']},
#     }
# )

test.core.database.sqltools.close_connection()

# filter_1, filter_2
var